In [1]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
from sklearn.utils import shuffle

In [2]:
data_path = r'/Users/petike/Documents/www/sandra-arato/ARRBHack-2018/_data'

In [56]:
df = pd.read_csv(os.path.join(data_path,'road_crash_locations.csv'))

In [57]:
df.columns

Index(['Crash_Ref_Number', 'Crash_Severity', 'Crash_Year', 'Crash_Month',
       'Crash_Day_Of_Week', 'Crash_Hour', 'Crash_Nature', 'Crash_Type',
       'Crash_Longitude_GDA94', 'Crash_Latitude_GDA94', 'Crash_Street',
       'Crash_Street_Intersecting', 'Loc_Suburb', 'Loc_Local_Government_Area',
       'Loc_Post_Code', 'Loc_Police_Division', 'Loc_Police_District',
       'Loc_Police_Region', 'Loc_Queensland_Transport_Region',
       'Loc_Main_Roads_Region', 'Loc_ABS_Statistical_Area_2',
       'Loc_ABS_Statistical_Area_3', 'Loc_ABS_Statistical_Area_4',
       'Loc_ABS_Remoteness', 'Loc_State_Electorate', 'Loc_Federal_Electorate',
       'Crash_Controlling_Authority', 'Crash_Roadway_Feature',
       'Crash_Traffic_Control', 'Crash_Speed_Limit',
       'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition',
       'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align',
       'Crash_Road_Vert_Align', 'Crash_DCA_Code', 'Crash_DCA_Description',
       'Crash_DCA_Group_Description', 'C

In [58]:
df.shape

(306558, 50)

In [65]:
cleaned = df[['Crash_Severity', 'Crash_Month', 'Crash_Day_Of_Week', 'Crash_Hour',
             'Crash_Longitude_GDA94', 'Crash_Latitude_GDA94',
             'Loc_Suburb',
             'Loc_ABS_Remoteness',
             'Crash_Roadway_Feature',
             'Crash_Traffic_Control', 'Crash_Speed_Limit',
             'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition',
             'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align', 'Crash_Road_Vert_Align',
             'Count_Casualty_Fatality',
             'Count_Casualty_Hospitalised', 'Count_Casualty_MedicallyTreated',
             'Count_Casualty_MinorInjury', 'Count_Casualty_Total', 'Count_Unit_Car',
             'Count_Unit_Motorcycle_Moped', 'Count_Unit_Truck', 'Count_Unit_Bus',
             'Count_Unit_Bicycle', 'Count_Unit_Pedestrian', 'Count_Unit_Other' 
             ]]

Removing rows without valid GPS coordinates

In [69]:
isValidLng = cleaned['Crash_Longitude_GDA94'] != 0
isValidLat = cleaned['Crash_Latitude_GDA94'] != 0
cleaned = cleaned[isValidLng][isValidLat]

In [70]:
cleaned.head()

,Crash_Severity,Crash_Month,Crash_Day_Of_Week,Crash_Hour,Crash_Longitude_GDA94,Crash_Latitude_GDA94,Loc_Suburb,Loc_ABS_Remoteness,Crash_Roadway_Feature,Crash_Traffic_Control,...,Count_Casualty_MedicallyTreated,Count_Casualty_MinorInjury,Count_Casualty_Total,Count_Unit_Car,Count_Unit_Motorcycle_Moped,Count_Unit_Truck,Count_Unit_Bus,Count_Unit_Bicycle,Count_Unit_Pedestrian,Count_Unit_Other
0,Property damage only,January,Monday,22,153.028793,-27.449659,Herston,Major cities,Intersection - T-Junction,Operating traffic lights,...,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Property damage only,January,Monday,2,152.966657,-27.049444,Caboolture,Major cities,No Roadway Feature,No traffic control,...,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Medical treatment,January,Monday,14,152.821788,-27.573498,Karalee,Major cities,No Roadway Feature,No traffic control,...,1,0,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Minor injury,January,Monday,0,152.974398,-27.225505,Dakabin,Major cities,Intersection - T-Junction,No traffic control,...,0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Medical treatment,January,Monday,15,153.002385,-27.248969,Kallangur,Major cities,Intersection - Cross,Operating traffic lights,...,1,0,1,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
cleaned.shape

(305846, 28)

Cleaning features with conditions data

In [75]:
isCrashRoadSurfaceConditionNotUnknown = cleaned['Crash_Road_Surface_Condition'] != 'Unknown'
isCrashAtmosphericConditionNotUnknown = cleaned['Crash_Atmospheric_Condition'] != 'Unknown'
isCrashLightingConditionNotUnknown = cleaned['Crash_Lighting_Condition'] != 'Unknown'
cleaned = cleaned[isCrashRoadSurfaceConditionNotUnknown & isCrashAtmosphericConditionNotUnknown & isCrashLightingConditionNotUnknown]

In [76]:
cleaned.shape

(305134, 28)

In [77]:
cleaned.head()

,Crash_Severity,Crash_Month,Crash_Day_Of_Week,Crash_Hour,Crash_Longitude_GDA94,Crash_Latitude_GDA94,Loc_Suburb,Loc_ABS_Remoteness,Crash_Roadway_Feature,Crash_Traffic_Control,...,Count_Casualty_MedicallyTreated,Count_Casualty_MinorInjury,Count_Casualty_Total,Count_Unit_Car,Count_Unit_Motorcycle_Moped,Count_Unit_Truck,Count_Unit_Bus,Count_Unit_Bicycle,Count_Unit_Pedestrian,Count_Unit_Other
1,Property damage only,January,Monday,2,152.966657,-27.049444,Caboolture,Major cities,No Roadway Feature,No traffic control,...,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Medical treatment,January,Monday,14,152.821788,-27.573498,Karalee,Major cities,No Roadway Feature,No traffic control,...,1,0,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Minor injury,January,Monday,0,152.974398,-27.225505,Dakabin,Major cities,Intersection - T-Junction,No traffic control,...,0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Medical treatment,January,Monday,15,153.002385,-27.248969,Kallangur,Major cities,Intersection - Cross,Operating traffic lights,...,1,0,1,3.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Minor injury,January,Monday,11,153.088907,-27.680827,Crestmead,Major cities,Intersection - Cross,Operating traffic lights,...,0,1,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0


Only dealing with incidents in major cities


In [79]:
isMajorCities = cleaned['Loc_ABS_Remoteness'] == 'Major cities'
cleaned = cleaned[isMajorCities]
cleaned.shape

(182492, 28)

In [81]:
cleaned.head()

,Crash_Severity,Crash_Month,Crash_Day_Of_Week,Crash_Hour,Crash_Longitude_GDA94,Crash_Latitude_GDA94,Loc_Suburb,Loc_ABS_Remoteness,Crash_Roadway_Feature,Crash_Traffic_Control,...,Count_Casualty_MedicallyTreated,Count_Casualty_MinorInjury,Count_Casualty_Total,Count_Unit_Car,Count_Unit_Motorcycle_Moped,Count_Unit_Truck,Count_Unit_Bus,Count_Unit_Bicycle,Count_Unit_Pedestrian,Count_Unit_Other
1,Property damage only,January,Monday,2,152.966657,-27.049444,Caboolture,Major cities,No Roadway Feature,No traffic control,...,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Medical treatment,January,Monday,14,152.821788,-27.573498,Karalee,Major cities,No Roadway Feature,No traffic control,...,1,0,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Minor injury,January,Monday,0,152.974398,-27.225505,Dakabin,Major cities,Intersection - T-Junction,No traffic control,...,0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Medical treatment,January,Monday,15,153.002385,-27.248969,Kallangur,Major cities,Intersection - Cross,Operating traffic lights,...,1,0,1,3.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Minor injury,January,Monday,11,153.088907,-27.680827,Crestmead,Major cities,Intersection - Cross,Operating traffic lights,...,0,1,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
cleaned['Crash_Road_Horiz_Align'].value_counts()

Straight                  148032
Curved - view open         28074
Curved - view obscured      6385
Unknown                        1
Name: Crash_Road_Horiz_Align, dtype: int64

In [84]:
cleaned['Crash_Road_Vert_Align'].value_counts()

Level      135678
Grade       30868
Crest        9797
Dip          6148
Unknown         1
Name: Crash_Road_Vert_Align, dtype: int64

Removing 'Unkown' values from Crash_Road_Horiz_Align and Crash_Road_Vert_Align

In [89]:
isHorizAlignKnown = cleaned['Crash_Road_Horiz_Align'] != 'Unknown'
isVertAlignKnown = cleaned['Crash_Road_Vert_Align'] != 'Unknown'
cleaned = cleaned[isHorizAlignKnown & isVertAlignKnown]
cleaned.shape

(182491, 28)

In [102]:
cleaned['Cyclist_FSI'] = cleaned.apply(lambda row: (row.Crash_Severity in ['Fatal', 'Hospitalisation'] and row.Count_Unit_Bicycle > 0), axis=1)


False    179066
True       3425
Name: Cyclist_FSI, dtype: int64

In [105]:
cleaned['Cyclist_FSI'].value_counts()

False    179066
True       3425
Name: Cyclist_FSI, dtype: int64

In [106]:
shuffled = shuffle(cleaned)

In [107]:
shuffled.to_csv(os.path.join(data_path,'cyclist_fsi_shuffled.csv'),
                          index=False,
                         columns=['Crash_Month', 'Crash_Day_Of_Week', 'Crash_Hour',
                                 'Crash_Longitude_GDA94', 'Crash_Latitude_GDA94',
                                 'Loc_Suburb',
                                 'Crash_Roadway_Feature',
                                 'Crash_Traffic_Control', 'Crash_Speed_Limit',
                                 'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition',
                                 'Crash_Lighting_Condition', 'Crash_Road_Horiz_Align', 'Crash_Road_Vert_Align',
                                 'Cyclist_FSI' 
                                 ])

In [108]:
cleaned['Cyclist_FSI'].value_counts()

False    179066
True       3425
Name: Cyclist_FSI, dtype: int64

In [109]:
shuffled[:10000]['Cyclist_FSI'].value_counts()

False    9810
True      190
Name: Cyclist_FSI, dtype: int64

,Crash_Hour,Crash_Longitude_GDA94,Crash_Latitude_GDA94,Cyclist_FSI
Crash_Hour,1.000000,0.011897,-0.012481,-0.029247
Crash_Longitude_GDA94,0.011897,1.000000,-0.528998,0.022021
Crash_Latitude_GDA94,-0.012481,-0.528998,1.000000,0.000617
Cyclist_FSI,-0.029247,0.022021,0.000617,1.000000
